In [1]:
from peewee import *
from random import randint
from typing import Any, Callable, TypeVar
import json

In [2]:
T = TypeVar('T')

In [3]:
def pad_start(width, fillchar, number) -> int:
    return int(str(number).rjust(width, fillchar))

def pad_end(width, fillchar, number) -> int:
    return int(str(number).ljust(width, fillchar))

In [4]:

def get_random_integer(min_value: T = 1, max_value: T = 9) -> int:
    min = pad_start(7,'1', pad_end(6, '0', min_value))
    max = pad_start(8, '9', pad_end(7, '9', max_value))
    return randint(min, max)

rand_number = get_random_integer()

print(rand_number)

72762654


In [5]:
database_name = 'notebooks_2_{}.db'.format(get_random_integer())
print(database_name)

notebooks_2_14980709.db


In [6]:
db = SqliteDatabase(database_name)

In [7]:
# dir(db)

In [8]:
db.connection();

In [9]:
class BaseModel(Model):
    class Meta:
        database = db

In [10]:
class Person(BaseModel):
    id = AutoField()
    first_name = CharField()
    last_name = CharField()
    phone = CharField(null=True)
    
class Experience(BaseModel):
    id = AutoField()
    title = CharField()
    company = CharField()
    start_date = DateField()
    end_date = DateField(null=True)
    person = ForeignKeyField(Person, backref='experiences')
    
class Contact(BaseModel):
    id = AutoField()
    nickname = CharField()
    owner = ForeignKeyField(Person, backref='contacts')

class Phone(BaseModel):
    id = AutoField()
    type = CharField()
    number = CharField()
    contact = ForeignKeyField(Contact, backref='phones')


In [11]:
db.create_tables([Person, Experience, Contact, Phone])

In [12]:
data = {
    'person_record': {
        'first_name': 'Gabriel',
        'last_name': 'Ariza',
        'phone': '1234567890'
    },
    'experience_records': [
        {
            'title'      :      'Developer',
            'company'    :      'Google',
            'start_date' :      '2018-01-01',
            'end_date'   :      '2018-02-01',
            'person_id'     :      0
        }
    ],
    'contact_records': [
        {
            'nickname': 'john_doe',
            'owner_id': 0
        }
    ],

    'phone_records': [
        {
            'type': 'home',
            'number': '1234567890',
            'contact_id': 0
        }
    ]
}


In [13]:
insert_data = data.get('person_record', {})
print(json.dumps([insert_data], indent=4))

[
    {
        "first_name": "Gabriel",
        "last_name": "Ariza",
        "phone": "1234567890"
    }
]


In [14]:
insert_response = Person.create(**insert_data)
print(json.dumps([insert_response.__data__], indent=4))
# dir(insert_response)

[
    {
        "first_name": "Gabriel",
        "last_name": "Ariza",
        "phone": "1234567890",
        "id": 1
    }
]


In [15]:
insert_data = data.get('person_record', {})
insert_data['experiences'] = data.get('experience_records', [])
insert_data['contacts'] = data.get('contact_records', [])
for i, contact in enumerate(insert_data['contacts']):
    contact.update({'nickname': '{}. {}'.format((i + 1), contact.get('nickname', ''))})
    contact['phones'] = data.get('phone_records', [])

print(json.dumps([insert_data], indent=4))

[
    {
        "first_name": "Gabriel",
        "last_name": "Ariza",
        "phone": "1234567890",
        "experiences": [
            {
                "title": "Developer",
                "company": "Google",
                "start_date": "2018-01-01",
                "end_date": "2018-02-01",
                "person_id": 0
            }
        ],
        "contacts": [
            {
                "nickname": "1. john_doe",
                "owner_id": 0,
                "phones": [
                    {
                        "type": "home",
                        "number": "1234567890",
                        "contact_id": 0
                    }
                ]
            }
        ]
    }
]


In [16]:
# The create method should insert the person record into the database
insert_response = Person.create(**insert_data)

In [17]:
print(type(insert_response)) # Should to be the class model: Person <Model: Person>
print(insert_response) # Should to output the person primary key (id) value (2)
print(insert_response.id) # Should be the same value as above as the both are outputting the primary key (id) value (2)

<Model: Person>
2
2


In [18]:
print(json.dumps([insert_response.__data__], indent=2))

[
  {
    "first_name": "Gabriel",
    "last_name": "Ariza",
    "phone": "1234567890",
    "id": 2
  }
]


In [19]:
print(json.dumps([insert_response.experiences], indent=2))

[
  [
    {
      "title": "Developer",
      "company": "Google",
      "start_date": "2018-01-01",
      "end_date": "2018-02-01",
      "person_id": 0
    }
  ]
]


In [20]:
for exp in insert_response.experiences:
    exp.update({'person_id': insert_response.id})

In [21]:
print(json.dumps(insert_response.experiences, indent=2))

[
  {
    "title": "Developer",
    "company": "Google",
    "start_date": "2018-01-01",
    "end_date": "2018-02-01",
    "person_id": 2
  }
]


In [22]:
print(json.dumps(insert_response.contacts, indent=4))

[
    {
        "nickname": "1. john_doe",
        "owner_id": 0,
        "phones": [
            {
                "type": "home",
                "number": "1234567890",
                "contact_id": 0
            }
        ]
    }
]


In [23]:
for contact in insert_response.contacts:
    contact.update({'owner_id': insert_response.id})

In [24]:
print(json.dumps(insert_response.contacts, indent=4))

[
    {
        "nickname": "1. john_doe",
        "owner_id": 2,
        "phones": [
            {
                "type": "home",
                "number": "1234567890",
                "contact_id": 0
            }
        ]
    }
]


In [25]:
experiencesEntity: list[Experience] = []
for index, exp in enumerate(insert_response.experiences):
    experiencesEntity.append(Experience(**exp))

In [26]:
print(type(experiencesEntity[0]))

<Model: Experience>


In [27]:
print(type(experiencesEntity[0].person))

<Model: Person>


In [28]:
print(json.dumps([experiencesEntity[0].person.__data__], indent=4))

[
    {
        "id": 2,
        "first_name": "Gabriel",
        "last_name": "Ariza",
        "phone": "1234567890"
    }
]


In [29]:
contactsEntity: list[Contact] = []
for index, contact in enumerate(insert_response.contacts):
    contactsEntity.append(Contact(**contact))

In [30]:
print(type(contactsEntity[0]))

<Model: Contact>


In [31]:
print(type(contactsEntity[0].owner))

<Model: Person>


In [32]:

try:
    insert_response.__data__['experiences'] = insert_response.experiences
    insert_response.__data__['contacts'] = insert_response.contacts
except Exception as e:
    print(e)

print(json.dumps([insert_response.__data__], indent=4))
dir(insert_response)

[
    {
        "first_name": "Gabriel",
        "last_name": "Ariza",
        "phone": "1234567890",
        "id": 2,
        "experiences": [
            {
                "title": "Developer",
                "company": "Google",
                "start_date": "2018-01-01",
                "end_date": "2018-02-01",
                "person_id": 2
            }
        ],
        "contacts": [
            {
                "nickname": "1. john_doe",
                "owner_id": 2,
                "phones": [
                    {
                        "type": "home",
                        "number": "1234567890",
                        "contact_id": 0
                    }
                ]
            }
        ]
    }
]


['DoesNotExist',
 '__class__',
 '__data__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__isabstractmethod__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__rel__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__sql__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_coerce',
 '_dirty',
 '_meta',
 '_normalize_data',
 '_pk',
 '_pk_expr',
 '_populate_unsaved_relations',
 '_prune_fields',
 '_schema',
 'add_index',
 'alias',
 'bind',
 'bind_ctx',
 'bulk_create',
 'bulk_update',
 'clone',
 'coerce',
 'contacts',
 'copy',
 'create',
 'create_table',
 'delete',
 'delete_by_id',
 'delete_instance',
 'dependencies',
 'dirty_fields',
 'drop_table',
 'experiences',
 'filter',
 'first_name',
 'get',
 'get_by_id',
 'get_id',
 'get_or_create',
 'get_or_none',
 'id',
 'index',
 'insert',
 'insert_from',
 'insert_many',


In [33]:

dir(insert_response.__data__)

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [34]:
print(json.dumps([{'{}'.format(i):d} for i, d in insert_response.__data__.items()], indent=2))

[
  {
    "first_name": "Gabriel"
  },
  {
    "last_name": "Ariza"
  },
  {
    "phone": "1234567890"
  },
  {
    "id": 2
  },
  {
    "experiences": [
      {
        "title": "Developer",
        "company": "Google",
        "start_date": "2018-01-01",
        "end_date": "2018-02-01",
        "person_id": 2
      }
    ]
  },
  {
    "contacts": [
      {
        "nickname": "1. john_doe",
        "owner_id": 2,
        "phones": [
          {
            "type": "home",
            "number": "1234567890",
            "contact_id": 0
          }
        ]
      }
    ]
  }
]


In [35]:
print(json.dumps(insert_response.experiences, indent=4))

[
    {
        "title": "Developer",
        "company": "Google",
        "start_date": "2018-01-01",
        "end_date": "2018-02-01",
        "person_id": 2
    }
]


In [36]:
print(json.dumps(insert_response.contacts, indent=4))

[
    {
        "nickname": "1. john_doe",
        "owner_id": 2,
        "phones": [
            {
                "type": "home",
                "number": "1234567890",
                "contact_id": 0
            }
        ]
    }
]


In [37]:
print(insert_response.id)

2


In [38]:
experiences = Experience().select() # no experience records should exist at this moment

In [39]:
print([exp for exp in experiences]) # Should output an empty array

[]


In [40]:
insert_response.experiences = experiencesEntity
insert_response.contacts = contactsEntity

In [41]:
print(insert_response.experiences[0].person.first_name)
print(insert_response.contacts[0].owner.first_name)

Gabriel
Gabriel


In [42]:
experience_bulk_create = Experience.bulk_create(insert_response.experiences)

In [46]:
experiences = Experience().select() # The person experiences should be retrived

In [47]:
print(len(experiences) )

1


In [48]:
contacts_bulk_create = Contact.bulk_create(insert_response.contacts)

In [50]:
contacts = Contact.select() # The person contacts should be retrived

In [51]:
print(len(contacts))

1


In [52]:
print(len(insert_response.contacts[0].phones))
print(len(contacts[0].phones))

1
0


In [53]:
print(insert_response.contacts[0].save())

1


In [54]:
print(insert_response.contacts[0].id)
print(insert_response.contacts[0].nickname)
print(insert_response.contacts[0].phones[0])


2
1. john_doe
{'type': 'home', 'number': '1234567890', 'contact_id': 0}


In [55]:
db.close()

True